# MNIST on Slim

Step 1: Import packages.

In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import input_data

Step 2: Prepare MNIST data.

In [2]:
mnist_dir = '../../data/MNIST_data'

if not os.path.exists(mnist_dir):
    print('Please prepare MNIST_data!')
    exit(1)

# find the target
mnist = input_data.read_data_sets(mnist_dir,one_hot=True)
X_test = mnist.test.images
y_test = mnist.test.labels
len_test = len(y_test)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Step 3: Build models.

In [3]:
features = tf.placeholder(tf.float32,[None,784])
labels = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

inputs_reshaped = tf.reshape(features, [-1, 28, 28, 1])
# layer1: cnn
conv1 = slim.conv2d(inputs_reshaped, num_outputs=32, kernel_size=[5, 5], padding='SAME', scope="layer1_conv")
conv1_pooling = slim.max_pool2d(conv1, kernel_size=[2, 2],stride=2,scope="layer1_pooling")
# layer2: cnn
conv2 = slim.conv2d(conv1_pooling, num_outputs=64, kernel_size=[5, 5], padding='SAME', scope="layer2_conv")
conv2_pooling = slim.max_pool2d(conv2, kernel_size=[2, 2],stride=2,scope="layer2_pooling")

# reshape
conv2_reshape = slim.flatten(conv2_pooling, scope="reshape")

# layer3: nn
nn3 = slim.fully_connected(conv2_reshape, num_outputs=500, activation_fn=tf.nn.sigmoid, scope="layer3") # activation_fn is default to ReLU
# layer4: nn
#nn4 = slim.fully_connected(nn3, num_outputs=10, scope="layer4")
#y_predicted = slim.fully_connected(nn3, num_outputs=10, activation_fn=tf.nn.softmax, scope="layer4")
y_predicted = slim.fully_connected(nn3, num_outputs=10, activation_fn=None, scope="layer4")

#y_predicted = nn4

####################################################################################################################
# Do not use the loss above!! Otherwise, the params will not be updated
####################################################################################################################
#loss=-tf.reduce_mean(tf.reduce_sum(labels*tf.log(y_predicted)))
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_predicted, labels=tf.argmax(labels, 1)))

#train=tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
train=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

Step 4: Train models.

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1000):
        X_train,y_train = mnist.train.next_batch(100)
        sess.run(train,feed_dict = {features:X_train,labels:y_train,keep_prob:0.3})

        # check accuracy
        if epoch % 50 == 0:
            y_pre = sess.run(y_predicted,feed_dict={features:X_test,labels:y_test,keep_prob:1})
            total = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(y_pre,axis=1),tf.argmax(y_test,axis=1)),tf.int16))
            print("epoch {0}: {1}/{2}".format(epoch,sess.run(total,feed_dict={features:X_test,labels:y_test,keep_prob:1}),len_test))


epoch 0: 1032/10000
epoch 50: 1953/10000
epoch 100: 2077/10000
epoch 150: 2778/10000
epoch 200: 3004/10000
epoch 250: 3431/10000
epoch 300: 6177/10000
epoch 350: 6102/10000
epoch 400: 6899/10000
epoch 450: 7680/10000
epoch 500: 8016/10000
epoch 550: 8388/10000
epoch 600: 8503/10000
epoch 650: 8626/10000
epoch 700: 8671/10000
epoch 750: 8805/10000
epoch 800: 8809/10000
epoch 850: 8907/10000
epoch 900: 8939/10000
epoch 950: 8944/10000
